# 预处理飞猪数据集

飞猪数据集说明：https://tianchi.aliyun.com/dataset/113649

首先 check 文件是否存在：

In [1]:
import os
import numpy as np
import pandas as pd

data_dir = '../data/fliggy'

files = ['user_profile.csv', 'item_profile.csv', 'user_item_behavior_history.csv']

for f in files:
    if not os.path.exists(os.path.join(data_dir, f)):
        raise FileNotFoundError(f'Not found file: {f} in directory {os.path.abspath(data_dir)}')

print('Check files over.')

Check files over.


In [2]:
USER_NAMES = ['UserID', 'Age', 'Gender', 'Occupation', 'UserCity', 'uLabel']
ITEM_NAMES = ['ItemID', 'CateID', 'Item_city', 'iLabel']
BEHAVIOR_NAMES = ['UserID', 'ItemID', 'BehaviorType', 'TimeStamp']

In [3]:
def mapped2sequential(df: pd.DataFrame, columns: list, start_from_1=True):
    for c in columns:
        m = {}
        sign = 1 if start_from_1 else 0
        if isinstance(c, str):
            if c in df.columns:
                l = df[c].unique().tolist()
                for v in l:
                    if m.setdefault(v, sign) == sign:
                        sign += 1
                df[c] = df[c].map(m)
        elif isinstance(c, list):
            for sub_c in c:
                if isinstance(sub_c, str):
                    if sub_c in df.columns:
                        l = df[sub_c].unique().tolist()
                        for v in l:
                            if m.setdefault(v, sign) == sign:
                                sign += 1
                        df[sub_c] = df[sub_c].map(m)
                else:
                    raise ValueError('最多支持二级list')


def min_max_normalize(df: pd.DataFrame, columns: list):
    for c in columns:
        if c in df.columns:
            df[c] = (df[c] - df[c].min()) / (df[c].max() - df[c].min())


def std_normalize(df: pd.DataFrame, columns: list):
    for c in columns:
        if c in df.columns:
            df[c] = (df[c] - df[c].mean()) / df[c].std()


def multi_value_process(df: pd.DataFrame, column: str, sep: str):
    uMap, ans = {}, []
    for arr in df[column]:
        que = []
        for i in arr.split(sep):
            if uMap.get(i) is None:
                uMap[i] = len(uMap) + 1
            que.append(uMap[i])
        ans.append(que)
    return ans, len(uMap) + 1

读取数据函数

In [4]:
def read_data(file: str, sample_size, sep, names=None, dtype=None):
    if not os.path.exists(file):
        e = f'The file: {file} not exists.'
        raise FileNotFoundError(e)
    df = pd.read_csv(file, iterator=True, names=names, sep=sep, dtype=dtype)
    if sample_size > 0:
        df = df.get_chunk(sample_size)
    else:
        df = df.get_chunk()
    return df

开始读取数据

In [5]:
# 两亿条数据太多了，这里只加载五千万条
sample_num = 5e7
SEP = ','

users = read_data(f'{data_dir}/{files[0]}', -1, SEP, USER_NAMES)
items = read_data(f'{data_dir}/{files[1]}', -1, SEP, ITEM_NAMES)
behavior = read_data(f'{data_dir}/{files[2]}', sample_num, SEP, BEHAVIOR_NAMES)

观察 behavior 数据集，会发现其含有 UserID	ItemID	BehaviorType 完全一致，但是只有 TimeStamp 不同的记录

In [6]:
behavior.head(10)

,UserID,ItemID,BehaviorType,TimeStamp
0,2499531,264562,clk,1559587204
1,2499531,264562,clk,1559587234
2,2499531,264562,clk,1559587325
3,3744925,23419,clk,1559641606
4,3744925,23419,clk,1559641305
5,3744925,23419,clk,1559641392
6,3744925,23419,clk,1559640610
7,3744925,23419,clk,1559641105
8,3744925,23419,clk,1559640580
9,3744925,23419,clk,1559640564


因此合并三者都相同的项，使用 BehaviorCount 来表示交互的次数

In [7]:
behavior = behavior.drop(columns=['TimeStamp'])
behavior['BehaviorCount'] = 0
behavior = behavior.groupby(['UserID', 'ItemID', 'BehaviorType']).count().reset_index()

将三者数据集进行合并

In [8]:
# 直接进行合并
behavior = behavior.merge(users, on='UserID').merge(items, on='ItemID')
behavior.head(10)

,UserID,ItemID,BehaviorType,TimeStamp,BehaviorCount,Age,Gender,Occupation,UserCity,uLabel,CateID,Item_city,iLabel
0,1,269,clk,3,3,77,2,5,392,19;11;13;20,38,221,-1
1,35036,269,clk,3,3,120,2,11,267,11;14;17;16,38,221,-1
2,2688687,269,clk,3,3,63,2,9,267,13;14;7;16,38,221,-1
3,3111981,269,clk,6,6,61,3,11,279,5;14;21;16,38,221,-1
4,3359256,269,cart,5,5,8,3,3,288,13;12;20;16,38,221,-1
5,3359256,269,clk,6,6,8,3,3,288,13;12;20;16,38,221,-1
6,3746177,269,clk,3,3,54,2,6,57,19;12;20;6,38,221,-1
7,3898389,269,clk,2,2,115,2,3,508,19;5;20;6,38,221,-1
8,3937841,269,clk,1,1,33,2,4,562,19;14;7;6,38,221,-1
9,4012828,269,clk,1,1,13,3,4,224,19;3;14;17,38,221,-1


In [9]:
min_max_normalize(behavior, ['TimeStamp'])
mapped2sequential(behavior, ['UserID', 'ItemID', 'Occupation', 'CateID', 'BehaviorType', ['UserCity', 'Item_city']])
behavior.head(10)

,UserID,ItemID,BehaviorType,TimeStamp,BehaviorCount,Age,Gender,Occupation,UserCity,uLabel,CateID,Item_city,iLabel
0,1,1,1,0.000155,3,77,2,1,1,19;11;13;20,1,126,-1
1,2,1,1,0.000155,3,120,2,2,2,11;14;17;16,1,126,-1
2,3,1,1,0.000155,3,63,2,3,2,13;14;7;16,1,126,-1
3,4,1,1,0.000389,6,61,3,2,3,5;14;21;16,1,126,-1
4,5,1,2,0.000311,5,8,3,4,4,13;12;20;16,1,126,-1
5,5,1,1,0.000389,6,8,3,4,4,13;12;20;16,1,126,-1
6,6,1,1,0.000155,3,54,2,5,5,19;12;20;6,1,126,-1
7,7,1,1,0.000078,2,115,2,4,6,19;5;20;6,1,126,-1
8,8,1,1,0.000000,1,33,2,6,7,19;14;7;6,1,126,-1
9,9,1,1,0.000000,1,13,3,6,8,19;3;14;17,1,126,-1


In [10]:
# 处理多值属性
uLabel, u_label_vocab = multi_value_process(behavior, 'uLabel', ';')
iLabel, i_label_vocab = multi_value_process(behavior, 'iLabel', ';')

In [11]:
# 替换属性
behavior['uLabels'] = uLabel
behavior['iLabels'] = iLabel
behavior = behavior.drop(columns=['uLabel', 'iLabel'])
behavior.head(10)

,UserID,ItemID,BehaviorType,TimeStamp,BehaviorCount,Age,Gender,Occupation,UserCity,CateID,Item_city,uLabels,iLabels
0,1,1,1,0.000155,3,77,2,1,1,1,126,"[1, 2, 3, 4]",[1]
1,2,1,1,0.000155,3,120,2,2,2,1,126,"[2, 5, 6, 7]",[1]
2,3,1,1,0.000155,3,63,2,3,2,1,126,"[3, 5, 8, 7]",[1]
3,4,1,1,0.000389,6,61,3,2,3,1,126,"[9, 5, 10, 7]",[1]
4,5,1,2,0.000311,5,8,3,4,4,1,126,"[3, 11, 4, 7]",[1]
5,5,1,1,0.000389,6,8,3,4,4,1,126,"[3, 11, 4, 7]",[1]
6,6,1,1,0.000155,3,54,2,5,5,1,126,"[1, 11, 4, 12]",[1]
7,7,1,1,0.000078,2,115,2,4,6,1,126,"[1, 9, 4, 12]",[1]
8,8,1,1,0.000000,1,33,2,6,7,1,126,"[1, 5, 8, 12]",[1]
9,9,1,1,0.000000,1,13,3,6,8,1,126,"[1, 13, 5, 6]",[1]


为每个用户记录其所有交互过的 item

In [16]:
item_list_per_user = behavior.groupby(['UserID'])['ItemID'].apply(list).reset_index()
item_list_per_user.head(10)

,UserID,ItemID
0,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]"
1,2,"[1, 58, 316, 323, 558, 951, 1224, 1224, 1920, ..."
2,3,"[1, 294, 317, 348, 386, 726, 970, 974, 1873, 1..."
3,4,"[1, 430, 638, 639, 639, 640, 1842, 1941, 1941,..."
4,5,"[1, 1, 881, 881, 1193, 1193, 1525, 1535, 1540,..."
5,6,"[1, 31, 2662, 2906, 10620, 13157, 14977]"
6,7,"[1, 3058, 21231]"
7,8,"[1, 31, 185, 369, 370, 451, 552, 684, 906, 137..."
8,9,"[1, 15, 34, 56, 146, 196, 612, 635, 647, 659, ..."
9,10,"[2, 273, 336, 366, 1039, 1772, 2215, 3254, 399..."


In [17]:
item_list_per_user.columns = ['UserID', 'InteractItems']

In [20]:
behavior = behavior.merge(item_list_per_user, on='UserID')

In [23]:
query_col = USER_NAMES + ['BehaviorType']

,UserID,ItemID,BehaviorType,TimeStamp,BehaviorCount,Age,Gender,Occupation,UserCity,CateID,Item_city,uLabels,iLabels,InteractItems
0,1,1,1,0.000155,3,77,2,1,1,1,126,"[1, 2, 3, 4]",[1],"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]"
1,1,2,1,0.000000,1,77,2,1,1,2,13,"[1, 2, 3, 4]","[2, 3]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]"
2,1,3,1,0.000389,6,77,2,1,1,2,11,"[1, 2, 3, 4]",[1],"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]"
3,1,4,1,0.000000,1,77,2,1,1,3,1,"[1, 2, 3, 4]",[1],"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]"
4,1,5,1,0.000000,1,77,2,1,1,4,37,"[1, 2, 3, 4]",[1],"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]"
5,1,6,1,0.000078,2,77,2,1,1,3,1,"[1, 2, 3, 4]",[1],"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]"
6,1,7,1,0.000622,9,77,2,1,1,2,12,"[1, 2, 3, 4]",[1],"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]"
7,1,8,1,0.000155,3,77,2,1,1,3,29,"[1, 2, 3, 4]",[1],"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]"
8,1,9,1,0.000233,4,77,2,1,1,3,13,"[1, 2, 3, 4]",[1],"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]"
9,1,10,1,0.000000,1,77,2,1,1,3,85,"[1, 2, 3, 4]",[1],"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]"
